# Texture-Only

Here we convert images to sketch drawings and scramble image tiles, removing any shape/border information by leaving out patches containing both in-lesion as well as out-lesion area. Morever we sample tiles equally, such that no information about the area / size of the lesion is preserved.

In [ ]:
import cv2
import os
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

def crop_center(img,cropx):
    y,x,c = img.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropx//2    
    return img[starty:starty+cropx, startx:startx+cropx, :]

def create_line_drawing_image(img, dataset_mean):
    kernel = np.array([
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        ], np.uint8)
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_dilated = cv2.dilate(img_gray, kernel, iterations=1)
    img_diff = cv2.absdiff(img_dilated, img_gray)
    contour = 255 - img_diff

    contour = contour[:, :, np.newaxis]
    contour = np.concatenate([contour, contour, contour], -1)

    # Multiply with image mean
    contour = contour - contour.min()
    contour = contour / contour.max()
    contour = contour * dataset_mean

    contour = contour - contour.min()
    contour = contour / contour.max()
    contour *= 255.0

    contour = contour.astype(np.uint8)

    contour = cv2.cvtColor(contour, cv2.COLOR_RGB2BGR)

    return contour

def scramble_texture(img_name, IMG_SIZE, CROP_SIZE, RASTER_SIZE, basepath, seg_basepath, dataset):

    if dataset == 'Derm7pt':
        dataset_mean = [0.7579, 0.6583, 0.5911]
    elif dataset == 'ISIC-NV-MEL':
        dataset_mean = [0.7024, 0.5214, 0.5224]
    elif dataset == 'ISIC_multiclass':
        dataset_mean = [0.6809, 0.5183, 0.5192]
    
    seg_path = os.path.join(seg_basepath, img_name)
    img_path = os.path.join(basepath, img_name)

    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = crop_center(img, CROP_SIZE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = create_line_drawing_image(img, dataset_mean)

    seg = cv2.imread(seg_path)
    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))
    seg = crop_center(seg, CROP_SIZE)
    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))

    # Rasterize segmentations
    in_rasters = []
    out_rasters = []

    sqrt_num_rasters = IMG_SIZE//RASTER_SIZE
    num_rasters = sqrt_num_rasters*sqrt_num_rasters

    for idy in range(sqrt_num_rasters):
        for idx in range(sqrt_num_rasters):
            y_start = idy * RASTER_SIZE
            y_end = (idy + 1) * RASTER_SIZE
            x_start = idx * RASTER_SIZE
            x_end = (idx + 1) * RASTER_SIZE

            seg_raster = seg[y_start:y_end, x_start:x_end, :]

            u = np.unique(seg_raster)
            if len(u) != 1:
                pass
            else:
                raster = img[y_start:y_end, x_start:x_end, :]
                if u == 0:
                    out_rasters.append(raster)
                elif u == 255:
                    in_rasters.append(raster)

    new_image = np.zeros_like(img)

    if len(in_rasters) != 0 and len(out_rasters) != 0:
        # Scramble Textures
        for idy in range(sqrt_num_rasters):
            for idx in range(sqrt_num_rasters):
                id = idy + idx

                if id % 2 == 0:
                    # Inner
                    raster = random.choice(in_rasters)
                else:
                    # Outer
                    raster = random.choice(out_rasters)

                y_start = idy * RASTER_SIZE
                y_end = (idy + 1) * RASTER_SIZE
                x_start = idx * RASTER_SIZE
                x_end = (idx + 1) * RASTER_SIZE
                
                new_image[y_start:y_end, x_start:x_end, :] = raster
    else:
        # Scramble image
        for idy in range(sqrt_num_rasters):
            for idx in range(sqrt_num_rasters):
                raster = random.choice(in_rasters + out_rasters)

                y_start = idy * RASTER_SIZE
                y_end = (idy + 1) * RASTER_SIZE
                x_start = idx * RASTER_SIZE
                x_end = (idx + 1) * RASTER_SIZE
                
                new_image[y_start:y_end, x_start:x_end, :] = raster
    
    return new_image


# Derm7pt

In [ ]:
lst_settings = [
    {
        'IMG_SIZE': 256,
        'CROP_SIZE': 200,
        'RASTER_SIZE': 32,
    }
]

for setting in lst_settings:
    IMG_SIZE = setting['IMG_SIZE']
    CROP_SIZE = setting['CROP_SIZE']
    RASTER_SIZE = setting['RASTER_SIZE']
    DEST_FOLDER = f'Derm7pt/augmentations/textures-only_IMGSize-{IMG_SIZE}_RasterSize-{RASTER_SIZE}_CropSize-{CROP_SIZE}'

    data = pd.read_csv('Derm7pt/original/meta/meta.csv')
    data = data['derm']

    for img_name in data:

        img = scramble_texture(
            img_name, 
            IMG_SIZE=IMG_SIZE, 
            CROP_SIZE=CROP_SIZE, 
            RASTER_SIZE=RASTER_SIZE,
            basepath='Derm7pt/original/images',
            seg_basepath='Derm7pt/segmentations/images',
            dataset='Derm7pt',
            )

        # Save processed image
        basedir = img_name.split('/')[0]
        basedir = os.path.join(DEST_FOLDER, basedir)
        current_filepath = os.path.join(DEST_FOLDER, img_name)
                
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        
        cv2.imwrite(current_filepath, img)

# ISIC

In [ ]:
lst_settings = [
    {
        'IMG_SIZE': 256,
        'CROP_SIZE': 200,
        'RASTER_SIZE': 32,
    }
]

for setting in lst_settings:
    IMG_SIZE = setting['IMG_SIZE']
    CROP_SIZE = setting['CROP_SIZE']
    RASTER_SIZE = setting['RASTER_SIZE']
    DEST_FOLDER = f'ISIC/augmentations/textures-only_IMGSize-{IMG_SIZE}_RasterSize-{RASTER_SIZE}_CropSize-{CROP_SIZE}'

    data = pd.read_csv('ISIC/csvs/multi-class_filtered.csv')
    data = data['image']

    for img_name in data:

        img_name = '/'.join(img_name.split('/')[6:])

        img = scramble_texture(
            img_name, 
            IMG_SIZE=IMG_SIZE, 
            CROP_SIZE=CROP_SIZE, 
            RASTER_SIZE=RASTER_SIZE,
            basepath='ISIC/original',
            seg_basepath='ISIC/segmentations',
            dataset='ISIC_multiclass'
            )

        # Save processed image
        basedir = '/'.join(img_name.split('/')[:-1])
        basedir = os.path.join(DEST_FOLDER, basedir)
        current_filepath = os.path.join(DEST_FOLDER, img_name)
                
        if not os.path.exists(basedir):
            os.makedirs(basedir)
        
        cv2.imwrite(current_filepath, img)

